Hello People 

As promissed i have found the data needed to recreate the plot called 3. in Data Vis Main Ideas

The data is subdivided into 2 pandas dataframes. 

--- Sector_times_df ---

All laps and lap times for all drivers. Firthermore the data have an indicator showing if this laps/sectortime was the fastest.

-- Driver       --> Nickname e.g VET, MAG

-- LapNumber    --> Integer (1,2,3,4)

-- Sector1Time  --> In format MM, SS, MSS

-- Sector2Time  --> In format MM, SS, MSS

-- Sector3Time  --> In format MM, SS, MSS

-- LapTime      --> In format MM, SS, MSS

-- FastestSector1   --> Binary indicator (0,1)

-- FastestSector2   --> Binary indicator (0,1)	

-- FastestSector3   --> Binary indicator (0,1)	

-- FastestLap       --> Binary indicator (0,1)


---- telemetry_df ----

-- Driver	    --> Nickname e.g VET, MAG

-- LapNumber	--> Integer (1,2,3,4)

-- Time	        --> In format MM, SS, MSS

-- Speed	    --> Integer

-- RPM	        --> Integer

-- Sector1Time	--> In format MM, SS, MSS

-- Sector2Time	--> In format MM, SS, MSS

-- Sector3Time	--> In format MM, SS, MSS

-- Sector1	    --> Binary indicator (0,1)

-- Sector2	    --> Binary indicator (0,1)

-- Sector3      --> Binary indicator (0,1)

In [ ]:
import fastf1 as ff1
import pandas as pd

# Load the race session for the 2018 Australian GP
race = ff1.get_session(2018, 'Australian Grand Prix', 'R')

# Load the session data (this is necessary for laps and telemetry)
race.load()

# Access all laps for the race using the laps attribute
laps = race.laps

# Extract driver, lap number, sector times, and the total lap time for each lap and driver
sector_times_df = laps[['Driver', 'LapNumber', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']]

# Update Sector1Time for LapNumber == 1 using LapTime - Sector2Time - Sector3Time
# In this dataset for some reason there is no information for the first sectortime.
condition = sector_times_df['LapNumber'] == 1
sector_times_df.loc[condition, 'Sector1Time'] = sector_times_df.loc[condition, 'LapTime'] - \
                                                 sector_times_df.loc[condition, 'Sector2Time'] - \
                                                 sector_times_df.loc[condition, 'Sector3Time']

# For some reason there where no information for this observation, therefore it was calculated.
condition2 = (sector_times_df['LapNumber'] == 26.0) & (sector_times_df['Driver'] == 'VET')
sector_times_df.loc[condition2, 'Sector3Time'] = sector_times_df.loc[condition2, 'LapTime'] - \
                                                 sector_times_df.loc[condition2, 'Sector1Time'] - \
                                                 sector_times_df.loc[condition2, 'Sector2Time']


# Create dummy columns to indicate the fastest times for each sector and lap time
sector_times_df['FastestSector1'] = (sector_times_df['Sector1Time'] == sector_times_df.groupby('Driver')['Sector1Time'].transform('min')).astype(int)
sector_times_df['FastestSector2'] = (sector_times_df['Sector2Time'] == sector_times_df.groupby('Driver')['Sector2Time'].transform('min')).astype(int)
sector_times_df['FastestSector3'] = (sector_times_df['Sector3Time'] == sector_times_df.groupby('Driver')['Sector3Time'].transform('min')).astype(int)
sector_times_df['FastestLap'] = (sector_times_df['LapTime'] == sector_times_df.groupby('Driver')['LapTime'].transform('min')).astype(int)


# Sort the data by Driver and LapNumber for better readability
sector_times_df = sector_times_df.sort_values(by=['Driver', 'LapNumber']).reset_index(drop=True)


In [2]:
# There are still some NA's for people who did not finish a round.
# i dont know if our plots can handle it NA's however in our case these missing observations actually carry some information.

# Get missing values.
nat_rows = sector_times_df[
    sector_times_df[['Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']].isna().any(axis=1)
]

# Count the number of times each driver appears (i.e., number of laps per driver)
driver_lap_counts = sector_times_df['Driver'].value_counts()
print(nat_rows[['Driver','LapNumber','Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']])
print(driver_lap_counts)

    Driver  LapNumber            Sector1Time            Sector2Time  \
121    ERI        6.0 0 days 00:00:30.840000 0 days 00:00:24.446000   
135    GAS       14.0 0 days 00:00:30.247000 0 days 00:00:24.226000   
707    SIR        5.0                    NaT                    NaT   

    Sector3Time LapTime  
121         NaT     NaT  
135         NaT     NaT  
707         NaT     NaT  
Driver
ALO    58
BOT    58
HUL    58
HAM    58
VER    58
VAN    58
OCO    58
LEC    58
PER    58
SAI    58
RIC    58
RAI    58
VET    58
STR    58
HAR    57
GRO    24
MAG    22
GAS    14
ERI     6
SIR     5
Name: count, dtype: int64


In [3]:
# Now get a dataframe which contains data on speed and RPM for each car and driver.
telemetry_data_list = []

# Loop through each lap to extract telemetry data for speed and RPM at each moment
for lap in laps.iterlaps():
    driver_lap = lap[1]  # Get the lap data
    telemetry = driver_lap.get_car_data()  # Get telemetry data for that lap
    
    # Extract speed and RPM at each point in the lap
    for index, row in telemetry.iterrows():
        telemetry_data_list.append({
            'Driver': driver_lap['Driver'],
            'LapNumber': driver_lap['LapNumber'],
            'Time': row['Time'],  # Timestamp of telemetry data
            'Speed': row['Speed'],
            'RPM': row['RPM']
        })

# Convert telemetry data list to a DataFrame
telemetry_df = pd.DataFrame(telemetry_data_list)

In [4]:
telemetry_df = pd.merge(telemetry_df, sector_times_df[['Driver', 'LapNumber', 'Sector1Time', 'Sector2Time', 'Sector3Time']],
                        on=['Driver', 'LapNumber'], 
                        how='left')

telemetry_df

,Driver,LapNumber,Time,Speed,RPM,Sector1Time,Sector2Time,Sector3Time
0,GAS,1.0,0 days 00:00:00.174000,0,11416,0 days 00:00:41.610000,0 days 00:00:25.495000,0 days 00:00:37.955000
1,GAS,1.0,0 days 00:00:00.414000,3,10016,0 days 00:00:41.610000,0 days 00:00:25.495000,0 days 00:00:37.955000
2,GAS,1.0,0 days 00:00:00.653000,15,8336,0 days 00:00:41.610000,0 days 00:00:25.495000,0 days 00:00:37.955000
3,GAS,1.0,0 days 00:00:00.893000,22,6656,0 days 00:00:41.610000,0 days 00:00:25.495000,0 days 00:00:37.955000
4,GAS,1.0,0 days 00:00:01.094000,28,6518,0 days 00:00:41.610000,0 days 00:00:25.495000,0 days 00:00:37.955000
...,...,...,...,...,...,...,...,...
369379,ERI,6.0,0 days 00:01:45.250000,58,3817,0 days 00:00:30.840000,0 days 00:00:24.446000,NaT
369380,ERI,6.0,0 days 00:01:45.451000,57,3703,0 days 00:00:30.840000,0 days 00:00:24.446000,NaT
369381,ERI,6.0,0 days 00:01:45.691000,56,3412,0 days 00:00:30.840000,0 days 00:00:24.446000,NaT
369382,ERI,6.0,0 days 00:01:45.931000,54,3404,0 days 00:00:30.840000,0 days 00:00:24.446000,NaT


In [5]:
import pandas as pd

# Define the function to assign sector dummies
def assign_sector_dummy(row):
    # Convert sector times to total seconds for comparison
    sector1_end = row['Sector1Time']
    sector2_end = sector1_end + row['Sector2Time']
    
    # Convert 'Time' to seconds
    time_in_seconds = row['Time']

    # Initialize dummies
    sector1_dummy = 0
    sector2_dummy = 0
    sector3_dummy = 0

    # Assign sector dummies based on the 'Time'
    if time_in_seconds < sector1_end:
        sector1_dummy = 1
    elif sector1_end <= time_in_seconds < sector2_end:
        sector2_dummy = 1
    else:
        sector3_dummy = 1

    # Return the dummies
    return pd.Series([sector1_dummy, sector2_dummy, sector3_dummy], index=['Sector1', 'Sector2', 'Sector3'])

# Apply the function to the DataFrame to create dummy variables for each observation
telemetry_df[['Sector1', 'Sector2', 'Sector3']] = telemetry_df.apply(assign_sector_dummy, axis=1)



In [6]:
# Function to format the timedelta back to MM:SS.fff
def format_timedelta_back(timedelta_obj):
    if pd.isna(timedelta_obj):
        return None
    total_seconds = timedelta_obj.total_seconds()
    minutes, seconds = divmod(total_seconds, 60)
    return f"{int(minutes):02}:{seconds:06.3f}"

# Apply the formatting function to 'Time', 'Sector1Time', 'Sector2Time', and 'Sector3Time'
telemetry_df['Time'] = telemetry_df['Time'].apply(format_timedelta_back)
telemetry_df['Sector1Time'] = telemetry_df['Sector1Time'].apply(format_timedelta_back)
telemetry_df['Sector2Time'] = telemetry_df['Sector2Time'].apply(format_timedelta_back)
telemetry_df['Sector3Time'] = telemetry_df['Sector3Time'].apply(format_timedelta_back)

In [7]:
# Apply the formatting function to 'Time', 'Sector1Time', 'Sector2Time', and 'Sector3Time'
sector_times_df['LapTime'] = sector_times_df['LapTime'].apply(format_timedelta_back)
sector_times_df['Sector1Time'] = sector_times_df['Sector1Time'].apply(format_timedelta_back)
sector_times_df['Sector2Time'] = sector_times_df['Sector2Time'].apply(format_timedelta_back)
sector_times_df['Sector3Time'] = sector_times_df['Sector3Time'].apply(format_timedelta_back)

In [8]:
telemetry_df

,Driver,LapNumber,Time,Speed,RPM,Sector1Time,Sector2Time,Sector3Time,Sector1,Sector2,Sector3
0,GAS,1.0,00:00.174,0,11416,00:41.610,00:25.495,00:37.955,1,0,0
1,GAS,1.0,00:00.414,3,10016,00:41.610,00:25.495,00:37.955,1,0,0
2,GAS,1.0,00:00.653,15,8336,00:41.610,00:25.495,00:37.955,1,0,0
3,GAS,1.0,00:00.893,22,6656,00:41.610,00:25.495,00:37.955,1,0,0
4,GAS,1.0,00:01.094,28,6518,00:41.610,00:25.495,00:37.955,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
369379,ERI,6.0,01:45.250,58,3817,00:30.840,00:24.446,None,0,0,1
369380,ERI,6.0,01:45.451,57,3703,00:30.840,00:24.446,None,0,0,1
369381,ERI,6.0,01:45.691,56,3412,00:30.840,00:24.446,None,0,0,1
369382,ERI,6.0,01:45.931,54,3404,00:30.840,00:24.446,None,0,0,1


In [9]:
sector_times_df

,Driver,LapNumber,Sector1Time,Sector2Time,Sector3Time,LapTime,FastestSector1,FastestSector2,FastestSector3,FastestLap
0,ALO,1.0,00:38.707,00:24.692,00:38.129,01:41.528,0,0,0,0
1,ALO,2.0,00:30.482,00:24.593,00:36.490,01:31.565,0,0,0,0
2,ALO,3.0,00:30.619,00:24.376,00:36.309,01:31.304,0,0,0,0
3,ALO,4.0,00:30.031,00:24.386,00:36.134,01:30.551,0,0,0,0
4,ALO,5.0,00:30.524,00:24.550,00:36.836,01:31.910,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
935,VET,54.0,00:29.011,00:23.289,00:34.411,01:26.711,0,0,0,0
936,VET,55.0,00:29.103,00:23.440,00:34.440,01:26.983,0,0,0,0
937,VET,56.0,00:29.154,00:23.391,00:34.691,01:27.236,0,0,0,0
938,VET,57.0,00:29.017,00:23.412,00:34.561,01:26.990,0,0,0,0
